In [1]:
# written to create files of positive and negative drugs
# based on DMES with drugs as rows and genes as columns
# Written 9-17-19 JLW

import pickle,os,csv
from collections import defaultdict
import pandas as pd
from functools import partial


dd_dic = partial(defaultdict, dict)

# I couldn't get this to work
#ddd_set = functools.partial(dd_dic,set)
#dd_set = functools.partial(defaultdict,set)
#d = defaultdict(partial(dd_set, dict))

In [2]:
negf = 'drugs_to_dmes_false_positives.txt'
posf = 'drugs_to_dmes_true_positive.txt'

In [3]:
dR = csv.DictReader(open(posf,'r'),delimiter='\t')
# collect data by DME
pos_d = defaultdict(dd_dic)
for r in dR:
    DME = r['Label DME']
    pathfx_phen = r['PathFX Phenotype']
    dbid = r['DrugBankID']
    dname = r['Drug Name']
    genes = r['Network genes assoc to phen']
    if pos_d[DME][dbid] == {}:
        pos_d[DME][dbid]=genes
    else:
        orig_genes = set(pos_d[DME][dbid].split(','))
        new_genes = set(genes.split(','))
        merge_genes = ','.join(list(orig_genes.union(new_genes)))
        pos_d[DME][dbid] = merge_genes # append the gene
        
neg_d = defaultdict(dd_dic)
dR = csv.DictReader(open(negf,'r'),delimiter='\t')
for r in dR:
    DME = r['False Positive DME']
    pathfx_phen = r['PathFX Phenotype']
    dbid = r['DrugBankID']
    dname = r['Drug Name']
    genes = r['Network genes assoc to phen']
    if neg_d[DME][dbid] == {}:
        neg_d[DME][dbid]=genes
    else:
        orig_genes = set(neg_d[DME][dbid].split(','))
        new_genes = set(genes.split(','))
        merge_genes = ','.join(list(orig_genes.union(new_genes)))
        neg_d[DME][dbid] = merge_genes # append the gene

In [4]:
pos_DMES = [k for k in pos_d]
neg_DMES = [k for k in neg_d]
all_DMES = set(pos_DMES).union(set(neg_DMES))
# format the data for output

for (dme,drug_dic) in pos_d.items():
    dme_name = dme.replace(' ','_')
    data = []
    # store and label positive drugs
    for (dbid,gene_str) in drug_dic.items():
        row_data = {'name':dbid,'label':'positive'}
        for g in gene_str.split(','):
            row_data[g]=1
        data.append(row_data)
    
    # store and label negative drugs if available
    if dme in neg_d:
        drug_dic = neg_d[dme]
        for (dbid,gene_str) in drug_dic.items():
            row_data = {'name':dbid,'label':'negative'}
            for g in gene_str.split(','):
                row_data[g]=1
            data.append(row_data)
            
    # merge into Python data frame
    df = pd.DataFrame(data)  
    df = df.fillna(0)
    
    # reorder the columns
    cols = df.columns.tolist()
    new_cols = ['name','label'] + [c for c in cols if c != 'name' and c != 'label']
    df = df[new_cols]
    
    if dme == 'Hepatic necrosis':
        print(df)

    outf = open('.'.join([dme_name,'txt']),'w')
    df.to_csv(outf,sep='\t',index=False)
    

       name     label  CNR2  CXCR2  CYP3A4  FPR1  HADH  NDUFA10  NDUFA12  \
0   DB01189  positive   0.0    0.0     0.0   0.0   1.0      1.0      1.0   
1   DB01236  positive   0.0    0.0     1.0   0.0   1.0      1.0      1.0   
2   DB00956  positive   1.0    1.0     1.0   1.0   0.0      0.0      0.0   
3   DB00825  positive   1.0    1.0     0.0   1.0   0.0      0.0      0.0   
4   DB00997  negative   0.0    0.0     1.0   0.0   1.0      1.0      1.0   
5   DB04835  negative   1.0    1.0     1.0   1.0   0.0      0.0      0.0   
6   DB01192  negative   1.0    1.0     1.0   1.0   0.0      0.0      0.0   
7   DB00486  negative   1.0    1.0     0.0   1.0   0.0      0.0      0.0   
8   DB01220  negative   0.0    0.0     1.0   0.0   0.0      0.0      0.0   
9   DB01081  negative   1.0    1.0     0.0   1.0   0.0      0.0      0.0   
10  DB08816  negative   1.0    1.0     1.0   1.0   0.0      0.0      0.0   
11  DB01136  negative   0.0    0.0     1.0   0.0   1.0      0.0      1.0   
12  DB00708 